# Example Datashader Notebook

In this workbook, some cells are left blank for you to complete.

In the image of the original paper, there are nearly a **million** points to be displayed (983,041). 

If you try to use Matplotlib or Bokeh to display them, the notebook will timeout before the image displays as it would take so long.

Here we will show how to use the (relatively) new Datashader module to recreate it.

To see how to use python's native Image module to recreate the image from the paper, see

https://jeremyallenjacobson.github.io/ZerosReproduction/PoonenOdlyzkoZerosFigureReproducedPython.html

As we are interested in recreating the image of the paper, first we create a list of all possible coefficients $(a_n, \cdots, a_1, 1)$ of polynomials of the form
$$P(z)=a_nz^n+\cdots + a_1z + 1 $$
where $n\leq 16$ and $a_i\in\{0,1\}$
We use **pdmax** to indicate maximum degree (16).

In [ ]:
from itertools import product

In [ ]:
pdmax = 16
coefficientsLessEqual16 = list(product(range(2), repeat = pdmax ))
for k in range(len(coefficientsLessEqual16)):
    coefficientsLessEqual16[k] = coefficientsLessEqual16[k] + tuple([1])

In [ ]:
coefficientsLessEqual16

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def CreateZeros(coefficients):
    zeros = []
    for j in range(len(coefficients)):
        roots = np.roots(coefficients[j])
        for k in range(len(roots)):
            x = roots[k].real
            y = roots[k].imag
            zeros.append((x,y))
    labels = ['real', 'imaginary']
    df = pd.DataFrame.from_records(zeros, columns=labels)
    return df

Next we create out dataframe of zeros using the function above and we time the runtime with **%time**

In [ ]:
%time ZerosLessEqual16 = CreateZeros(coefficientsLessEqual16)

To display the dataframe, run the cell below. The leftmost column shows the indexing. In particular, one finds how many zeros there are (983041).

In [ ]:
ZerosLessEqual16

The following cell imports datashader and other necessary modules. Datashader is capable of displaying a million points quickly in part because, the image you see is divided into "bins", and datashader "bins" the actual points based on some **reduction** specified. 

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In the code below, **ds.Canvas().points(dataframe goes here)** creates a "canvas", and the function **tf.shade()** essentially colors/shades it according to a default.

In [ ]:
%time tf.shade(ds.Canvas().points(ZerosLessEqual16, 'real', 'imaginary'))

Finally, thanks to the Bokeh extension of datashader called **InteractiveImage** it is possible to use work interactively with millions of points.

In [ ]:
import bokeh.plotting as bp
from datashader.bokeh_ext import InteractiveImage
bp.output_notebook()

In [ ]:
p = bp.figure(tools='pan,wheel_zoom,reset', x_range=(-1.5,1.5), y_range=(-1.5,1.5))

def image_callback(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(ZerosLessEqual16, 'real', 'imaginary')
    img = tf.shade(agg)
    return tf.spread(img)

InteractiveImage(p, image_callback)

Now you try. Create a new list of coefficients of degree less than or equal to 17. 

Then create the dataframe of zeros.

Finally, create the interactive plot.